In [3]:
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

This code is submitted as part of project 2 for the subject COMP90037 (Security Analytics) at the University of  Melbourne .
     
    -------------------------------------------
    COMP90037 Security Analytics - Project 2 
    Machine learning based Threat detection

    Author : Mohammed Ahsan Kollathodi 
    Student id: 1048942.
    

### Train

In [4]:
pd_train = pd.read_csv('train_data_VT_A1.csv', sep=',')

In [5]:
pd_train.head()

,num_total_packets,total_bytes,src_bytes,packets_in_Sec,bytes_total_in_Sec,Source_Bytes_Sec,protocol,src_ip,src_port,direction,dst_ip,dst_port,state,dsttype_service
0,-0.011855,-0.011502,-0.006085,-0.009940,-0.057686,0.005116,8.0,46986.0,25937.0,6.0,42900.0,15542.0,69.0,0.0
1,0.097927,0.011758,0.018442,-0.164602,-0.127541,-0.045789,8.0,46986.0,35462.0,8.0,22164.0,58516.0,170.0,0.0
2,1.934192,0.468640,1.468779,-0.163396,-0.126357,-0.044450,9.0,47020.0,23255.0,8.0,23896.0,39465.0,16.0,0.0
3,-0.012677,-0.011578,-0.006190,-0.164696,-0.127552,-0.045794,2.0,46790.0,62779.0,6.0,80493.0,23018.0,325.0,4.0
4,12.183318,5.892219,15.613691,-0.160947,-0.120492,-0.038956,8.0,70245.0,48066.0,6.0,22770.0,38021.0,196.0,0.0


In [6]:
# set the contamination parameter equal to 0.02 
isln_frst = IsolationForest(contamination=0.02)

In [7]:
isln_frst.fit(pd_train)

IsolationForest(contamination=0.02)

### Test

In [23]:
pd_test = pd.read_csv('test_data_VT_A1.csv', sep=',')

In [24]:
predict_test = isln_frst.predict(pd_test)

In [25]:
decision_function = isln_frst.decision_function(pd_test)

In [26]:
pd_test['Anomalies'] = predict_test
pd_test['Score'] = decision_function

In [27]:
pd_test['Anomalies'].value_counts()  # The value ""-1" in the result would correspond to the number of anomalies. 

-1    348477
Name: Anomalies, dtype: int64

In [28]:
#sns.pairplot(pd_test, hue = 'Anomalies')

In [29]:
Anomalies = pd_test.loc[pd_test['Anomalies'] == -1]

In [30]:
Anomalies.head()

,num_total_packets,duration,total_bytes,src_bytes,packets_in_Sec,bytes_total_in_Sec,Source_Bytes_Sec,protocol,src_ip,src_port,direction,dst_ip,dst_port,state,Anomalies,Score
0,-0.012266,2752.193850,-0.011517,-0.006107,-0.164696,-0.127552,-0.045794,9,129295,56079,3,22461,53522,16,-1,-0.122696
1,-0.012471,1846.957650,-0.011519,-0.006120,-0.164696,-0.127552,-0.045794,9,142254,45879,3,22461,53520,16,-1,-0.119257
2,-0.012677,2079.032480,-0.011550,-0.006211,-0.164696,-0.127552,-0.045794,9,102654,58934,3,22461,53520,16,-1,-0.117854
3,-0.012471,1533.286880,-0.011550,-0.006211,-0.164696,-0.127552,-0.045794,9,29182,32236,3,22461,53524,16,-1,-0.102994
4,4.870326,71.863629,5.733420,0.364877,-0.164696,-0.127552,-0.045794,9,157870,35339,3,22461,32909,16,-1,-0.191642


In [37]:
test_original = pd.read_csv('testdata_cleaned.csv', sep=',')

In [38]:
anomalies_feat_orig = test_original[test_original.index.isin(Anomalies.index)]

In [39]:
anomalies_feat_orig = anomalies_feat_orig.merge(Anomalies[['Score']], left_index = True, right_index =True)

In [40]:
anomalies_feat_orig.head(10)

,timestamp,duration,protocol,src_ip,src_port,direction,dst_ip,dst_port,state,srctype_service,dsttype_service,num_total_packets,total_bytes,src_bytes,packets_in_Sec,bytes_total_in_Sec,Source_Bytes_Sec,Score
0,2021-08-15 18:01:29.175156,2752.193853,udp,74.225.127.74,60623,<->,150.35.87.62,63550,CON,0.0,0.0,3,431,288,0.001090,0.156602,0.104644,-0.122696
1,2021-08-15 18:01:42.659892,1846.957653,udp,81.237.57.248,51413,<->,150.35.87.62,63549,CON,0.0,0.0,2,421,270,0.001083,0.227942,0.146186,-0.119257
2,2021-08-15 18:01:44.669650,2079.032483,udp,34.150.123.142,63197,<->,150.35.87.62,63549,CON,0.0,0.0,1,290,145,0.000481,0.139488,0.069744,-0.117854
3,2021-08-15 18:15:52.811230,1533.286882,udp,121.8.38.67,39112,<->,150.35.87.62,63552,CON,0.0,0.0,2,289,145,0.001304,0.188484,0.094568,-0.102994
4,2021-08-15 18:24:13.357670,71.863629,udp,85.42.5.252,41912,<->,150.35.87.62,43085,CON,0.0,0.0,23753,24213065,507942,330.528814,336930.731094,7068.137364,-0.191642
5,2021-08-15 18:24:13.362772,28.076997,tcp,118.187.40.27,49189,<->,150.35.87.5,80,A_FPA,0.0,0.0,220,190900,6593,7.835596,6799.160121,234.818558,-0.166380
6,2021-08-15 18:24:13.364316,637.394983,tcp,83.81.82.159,51287,->,150.35.89.27,31004,FPA_FPA,0.0,0.0,15338,2513221,2234665,24.063572,3942.956984,3505.934404,-0.195270
7,2021-08-15 18:24:13.365134,0.000178,udp,85.76.247.59,39051,->,150.35.87.121,1154,INT,0.0,NaN,0,146,145,0.000000,819203.712438,813592.728107,-0.075552
8,2021-08-15 18:24:13.370725,452.563303,tcp,195.224.109.129,80,->,150.35.87.62,2774,FPA_FA,0.0,0.0,85022,97665708,96281686,187.867641,215805.628216,212747.443893,-0.209427
9,2021-08-15 18:24:13.373200,123.595564,tcp,215.114.5.154,7999,->,150.35.89.138,3975,PA_FRA,0.0,0.0,4225,4167880,4096642,34.184075,33721.922154,33145.542246,-0.195270


In [43]:
import numpy as np

anomalies_feat_orig.loc[anomalies_feat_orig['packets_in_Sec'] != np.inf].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,duration,src_port,dst_port,srctype_service,dsttype_service,num_total_packets,total_bytes,src_bytes,packets_in_Sec,bytes_total_in_Sec,Source_Bytes_Sec,Score
count,330963,330963,330963,330929,291215,330963,330963,330963,313538,330963,330957,330963
mean,542.472,32278.6,22253.9,0.00481674,0.000109884,40.3886,25737.7,9775.63,1199.15,inf,inf,-0.0900234
std,1067.47,19836.6,2.09493e+06,0.112634,0.0179659,3893.67,2.80977e+06,923503,5290.4,nan,nan,0.0268802
min,0,0,0,0,0,0,59,0,0,0.0404022,0,-0.209427
25%,0.000732101,14375.5,13360,0,0,1,138,74,0.0209766,9.97618,5.57385,-0.121825
50%,0.001299,32238,13362,0,0,2,401,146,697.452,157614,75092.5,-0.0740942
75%,188.978,50329,13365,0,0,4,928,471,1729.43,318343,109948,-0.0727828
max,3649.84,65537,5.39024e+08,3,3,1.4842e+06,1.06155e+09,2.638e+08,1.0015e+06,inf,inf,-0.022424


In [47]:
anomalies_feat_orig['src_ip'].nunique()

207030

In [48]:
anomalies_feat_orig.to_csv('test_A1_iForest_0.02_VT_orig_feat.csv', sep=',')
                    

